In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error 
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import json
import random

## Data Preparation & Preprocessing

In [46]:
# Set seed for reproducibility
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

set_seed(42)

In [47]:
# Load the data
def load_data(file):
    # Load the data
    data = pd.read_csv('data/' + file + '.csv')
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)

    # round to 2 decimal places
    data = data.round(2) 

    return data

data = load_data('SP500')

In [48]:
# Preprocess the data (using min-max scaler)
def normalize(features):
    min_val = np.min(features)
    max_val = np.max(features)
    return (features - min_val) / (max_val - min_val)
    
features = np.array(data[['Close']])
scaled_data = normalize(features)

In [49]:
# Load hyperparameters
with open('configs/configs.json', 'r') as file:
    hyperparams = json.load(file)

In [50]:
# Create sequences
def create_sequences(data, input_size, num_steps):
    data = [np.array(data[i * input_size: (i + 1) * input_size]) 
       for i in range(len(data) // input_size)]

    # Split into groups of `num_steps`
    X = np.array([data[i: i + num_steps] for i in range(len(data) - num_steps)])
    y = np.array([data[i + num_steps] for i in range(len(data) - num_steps)])
    
    return X, y

num_steps = hyperparams['num_steps'] # Extract number of steps
input_size = hyperparams['input_size'] # Extract number of steps
test_split = hyperparams['test_split']
X, y = create_sequences(scaled_data, input_size, num_steps)

split = int((1 - test_split) * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

## LSTM

In [51]:
# Extract hyperparameters
def extract_hyperparams(hyperparams, model):
    num_units = hyperparams[model][model + '_units']
    num_layers = hyperparams[model][model + '_layers']
    dropout_rate = hyperparams[model]['dropout_rate']
    dense_units = hyperparams[model]['dense_units']
    batch_size = hyperparams[model]['batch_size']
    max_epochs = hyperparams[model]['max_epochs']
    use_early_stop = hyperparams[model]['use_early_stop']
    early_stop_patience = hyperparams[model]['early_stop_patience']
    train_needed = hyperparams[model]['pretrain'] # Whether to train the model

    return num_units, num_layers, dropout_rate, dense_units, batch_size, epochs, use_early_stop, early_stop_patience, train_needed

num_units, num_layers, dropout_rate, dense_units, batch_size, max_epochs, use_early_stop, early_stop_patience, train_needed = extract_hyperparams(hyperparams, 'lstm')

In [ ]:
# Build (or load) model

In [ ]:
# import numpy as np
# from scipy.special import kl_div

# # Sample data for demonstration purposes
# # In practice, you would use actual activation data from the model
# def get_fp32_activation_data():
#     # Replace this with actual data collection
#     return np.array([0.5, 1.0, 2.0, -1.0, -0.5, 3.0, -3.0, 0.1])

# # **1. Calibration: Collect Data and Compute Scale Factor**

# # Collect activation data (replace with actual FP32 data)
# activation_data_fp32 = get_fp32_activation_data()

# # Step 1: Determine the maximum absolute value
# max_abs_value = np.max(np.abs(activation_data_fp32))

# # Step 2: Compute the scale factor for symmetric quantization
# def compute_scale(max_abs_value):
#     return max_abs_value / 127  # 127 because 8-bit quantization uses values from -128 to 127

# scale = compute_scale(max_abs_value)

# # Step 3: Perform preliminary quantization of FP32 data
# def preliminary_quantize(fp32_data, scale):
#     return np.clip(np.round(fp32_data / scale), -128, 127).astype(np.int8)

# activation_data_int8 = preliminary_quantize(activation_data_fp32, scale)

# # Step 4: Compute histograms for FP32 and INT8
# def compute_histograms(fp32_data, int8_data):
#     hist_fp32, _ = np.histogram(fp32_data, bins=256, range=(-128, 127), density=True)
#     hist_int8, _ = np.histogram(int8_data, bins=256, range=(-128, 127), density=True)
#     return hist_fp32, hist_int8

# hist_fp32, hist_int8 = compute_histograms(activation_data_fp32, activation_data_int8)

# # Step 5: Compute KL divergence
# def compute_kl_divergence(hist_fp32, hist_int8):
#     # Adding a small constant to avoid log(0)
#     return np.sum(kl_div(hist_fp32 + 1e-8, hist_int8 + 1e-8))

# kl_divergence = compute_kl_divergence(hist_fp32, hist_int8)
# print(f"KL Divergence: {kl_divergence}")

# # **2. Quantization: Convert FP32 to INT8**

# # Final quantization of FP32 data
# def quantize_to_int8(fp32_data, scale):
#     return np.clip(np.round(fp32_data / scale), -128, 127).astype(np.int8)

# activation_data_int8_final = quantize_to_int8(activation_data_fp32, scale)

# # **3. INT32 Computations: Perform Layer Operations**

# # Example INT32 computation function
# def int32_computations(weights, activations, bias):
#     # Perform INT32 matrix multiplication and add bias
#     int32_result = np.dot(weights, activations) + bias
#     return int32_result

# # Sample weights and bias for demonstration
# weights = np.array([[1, -1], [2, 3]])
# bias = np.array([1, -1])

# # Perform INT32 computations
# int32_result = int32_computations(weights, activation_data_int8_final, bias)
# print(f"INT32 Computation Result: {int32_result}")

# # **4. Re-Quantization: Convert INT32 to INT8**

# # Re-quantization process
# def requantize(int32_activations, scale, zero_point, bias):
#     # Add bias and then requantize
#     int32_activations_with_bias = int32_activations + bias
#     return np.clip(np.round(int32_activations_with_bias * scale) + zero_point, -128, 127).astype(np.int8)

# # Assuming zero_point = 0 for symmetric quantization
# zero_point = 0

# # Re-quantize INT32 results to INT8
# activation_data_int8_requantized = requantize(int32_result, scale, zero_point, bias)
# print(f"Re-Quantized INT8 Data: {activation_data_int8_requantized}")

# # **5. De-Quantization: Convert INT8 Back to FP32**

# # De-quantization process
# def dequantize_to_fp32(int8_data, scale, zero_point):
#     return (int8_data - zero_point) * scale

# # Convert INT8 results back to FP32
# fp32_reconstructed_data = dequantize_to_fp32(activation_data_int8_requantized, scale, zero_point)
# print(f"De-Quantized FP32 Data: {fp32_reconstructed_data}")

## GRU

In [53]:
# Extract hyperparameters
num_units, num_layers, dropout_rate, dense_units, batch_size, max_epochs, use_early_stop, early_stop_patience, train_needed = extract_hyperparams(hyperparams, 'gru')

In [54]:
# Build (or load) model